# eICU Data Joining
---

Reading and joining all preprocessed parts of the eICU dataset from MIT with the data from over 139k patients collected in the US.

The main goal of this notebook is to prepare a single CSV document that contains all the relevant data to be used when training a machine learning model that predicts mortality, joining tables, filtering useless columns and performing imputation.

## Importing the necessary packages

In [ ]:
import os                                  # os handles directory/workspace changes
import numpy as np                         # NumPy to handle numeric and NaN operations
import yaml                                # Save and load YAML files

In [ ]:
# Debugging packages
import pixiedust                           # Debugging in Jupyter Notebook cells

In [ ]:
# Change to parent directory (presumably "Documents")
os.chdir("../../..")
# Path to the CSV dataset files
data_path = 'data/eICU/uncompressed/cleaned/'
# Path to the code files
project_path = 'code/eICU-mortality-prediction/'

In [ ]:
# Make sure that every large operation can be handled, by using the disk as an overflow for the memory
!export MODIN_OUT_OF_CORE=true
# Another trick to do with Pandas so as to be able to allocate bigger objects to memory
!sudo bash -c 'echo 1 > /proc/sys/vm/overcommit_memory'

In [ ]:
import modin.pandas as pd                  # Optimized distributed version of Pandas
# import pandas as pd
import data_utils as du                    # Data science and machine learning relevant methods

Allow pandas to show more columns:

In [ ]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

Set the random seed for reproducibility

In [ ]:
du.set_random_seed(42)

## Initializing variables

In [ ]:
dtype_dict = {'patientunitstayid': 'uint32',
              'gender': 'UInt8',
              'age': 'float32',
              'admissionheight': 'float32',
              'admissionweight': 'float32',
              'death_ts': 'Int32',
              'ts': 'int32',
              'cad': 'UInt8',
              'cancer': 'UInt8',
              # 'diagnosis_type_1': 'UInt64',
              # 'diagnosis_disorder_2': 'UInt64',
              # 'diagnosis_detailed_3': 'UInt64',
              # 'allergyname': 'UInt64',
              # 'drugallergyhiclseqno': 'UInt64',
              # 'pasthistoryvalue': 'UInt64',
              # 'pasthistorytype': 'UInt64',
              # 'pasthistorydetails': 'UInt64',
              # 'treatmenttype': 'UInt64',
              # 'treatmenttherapy': 'UInt64',
              # 'treatmentdetails': 'UInt64',
              # 'drugunit_x': 'UInt64',
              # 'drugadmitfrequency_x': 'UInt64',
              # 'drughiclseqno_x': 'UInt64',
              'drugdosage_x': 'float32',
              # 'drugadmitfrequency_y': 'UInt64',
              # 'drughiclseqno_y': 'UInt64',
              'drugdosage_y': 'float32',
              # 'drugunit_y': 'UInt64',
              'bodyweight_(kg)': 'float32',
              'oral_intake': 'float32',
              'urine_output': 'float32',
              'i.v._intake	': 'float32',
              'saline_flush_(ml)_intake': 'float32',
              'volume_given_ml': 'float32',
              'stool_output': 'float32',
              'prbcs_intake': 'float32',
              'gastric_(ng)_output': 'float32',
              'dialysis_output': 'float32',
              'propofol_intake': 'float32',
              'lr_intake': 'float32',
              'indwellingcatheter_output': 'float32',
              'feeding_tube_flush_ml': 'float32',
              'patient_fluid_removal': 'float32',
              'fentanyl_intake': 'float32',
              'norepinephrine_intake': 'float32',
              'crystalloids_intake': 'float32',
              'voided_amount': 'float32',
              'nutrition_total_intake': 'float32',
              # 'nutrition': 'UInt64',
              # 'nurse_treatments': 'UInt64',
              # 'hygiene/adls': 'UInt64',
              # 'activity': 'UInt64',
              # 'pupils': 'UInt64',
              # 'neurologic': 'UInt64',
              # 'secretions': 'UInt64',
              # 'cough': 'UInt64',
              'priorvent': 'UInt8',
              'onvent': 'UInt8',
              'noninvasivesystolic': 'float32',
              'noninvasivediastolic': 'float32',
              'noninvasivemean': 'float32',
              'paop': 'float32',
              'cardiacoutput': 'float32',
              'cardiacinput': 'float32',
              'svr': 'float32',
              'svri': 'float32',
              'pvr': 'float32',
              'pvri': 'float32',
              'temperature': 'float32',
              'sao2': 'float32',
              'heartrate': 'float32',
              'respiration': 'float32',
              'cvp': 'float32',
              'etco2': 'float32',
              'systemicsystolic': 'float32',
              'systemicdiastolic': 'float32',
              'systemicmean': 'float32',
              'pasystolic': 'float32',
              'padiastolic': 'float32',
              'pamean': 'float32',
              'st1': 'float32',
              'st2': 'float32',
              'st3': 'float32',
              'icp': 'float32',
              # 'labtypeid': 'UInt64',
              # 'labname': 'UInt64',
              # 'lab_units': 'UInt64',
              'lab_result': 'float32'}

Load the lists of one hot encoded columns:

In [ ]:
stream_adms_drug = open(f'{data_path}cat_feat_ohe_adms_drug.yaml', 'r')
stream_inf_drug = open(f'{data_path}cat_feat_ohe_inf_drug.yaml', 'r')
stream_med = open(f'{data_path}cat_feat_ohe_med.yaml', 'r')
stream_treat = open(f'{data_path}cat_feat_ohe_treat.yaml', 'r')
stream_diag = open(f'{data_path}cat_feat_ohe_diag.yaml', 'r')
stream_alrg = open(f'{data_path}cat_feat_ohe_alrg.yaml', 'r')
stream_past_hist = open(f'{data_path}cat_feat_ohe_past_hist.yaml', 'r')
stream_lab = open(f'{data_path}cat_feat_ohe_lab.yaml', 'r')
stream_patient = open(f'{data_path}cat_feat_ohe_patient.yaml', 'r')
stream_notes = open(f'{data_path}cat_feat_ohe_note.yaml', 'r')

In [ ]:
cat_feat_ohe_adms_drug = yaml.load(stream_adms_drug, Loader=yaml.FullLoader)
cat_feat_ohe_inf_drug = yaml.load(stream_inf_drug, Loader=yaml.FullLoader)
cat_feat_ohe_med = yaml.load(stream_med, Loader=yaml.FullLoader)
cat_feat_ohe_treat = yaml.load(stream_treat, Loader=yaml.FullLoader)
cat_feat_ohe_diag = yaml.load(stream_diag, Loader=yaml.FullLoader)
cat_feat_ohe_alrg = yaml.load(stream_alrg, Loader=yaml.FullLoader)
cat_feat_ohe_past_hist = yaml.load(stream_past_hist, Loader=yaml.FullLoader)
cat_feat_ohe_lab = yaml.load(stream_lab, Loader=yaml.FullLoader)
cat_feat_ohe_patient = yaml.load(stream_patient, Loader=yaml.FullLoader)
cat_feat_ohe_notes = yaml.load(stream_notes, Loader=yaml.FullLoader)

In [ ]:
cat_feat_ohe = du.utils.merge_dicts([cat_feat_ohe_adms_drug, cat_feat_ohe_inf_drug,
                                     cat_feat_ohe_med, cat_feat_ohe_treat,
                                     cat_feat_ohe_diag, cat_feat_ohe_alrg,
                                     cat_feat_ohe_past_hist, cat_feat_ohe_lab,
                                     cat_feat_ohe_patient, cat_feat_ohe_notes])

In [ ]:
ohe_columns = du.utils.merge_lists(list(cat_feat_ohe.values()))
ohe_columns = du.data_processing.clean_naming(ohe_columns, lower_case=False)

Add the one hot encoded columns to the dtypes dictionary, specifying them with type `UInt8`

In [ ]:
for col in ohe_columns:
    dtype_dict[col] = 'UInt8'

In [ ]:
dtype_dict

## Loading the data

### Patient information

In [ ]:
patient_df = pd.read_csv(f'{data_path}normalized/ohe/patient.csv', dtype=dtype_dict)
patient_df.head()

In [ ]:
du.search_explore.dataframe_missing_values(patient_df)

Remove rows that don't identify the patient:

In [ ]:
patient_df = patient_df[~patient_df.patientunitstayid.isnull()]

In [ ]:
du.search_explore.dataframe_missing_values(patient_df)

In [ ]:
patient_df.dtypes

In [ ]:
note_df = pd.read_csv(f'{data_path}normalized/ohe/note.csv', dtype=dtype_dict)
note_df.head()

Remove the uneeded 'Unnamed: 0' column:

In [ ]:
patient_df = patient_df.drop(columns='Unnamed: 0')
note_df = note_df.drop(columns='Unnamed: 0')

### Diagnosis

In [ ]:
diagns_df = pd.read_csv(f'{data_path}normalized/ohe/diagnosis.csv', dtype=dtype_dict)
diagns_df.head()

In [ ]:
alrg_df = pd.read_csv(f'{data_path}normalized/ohe/allergy.csv', dtype=dtype_dict)
alrg_df.head()

In [ ]:
past_hist_df = pd.read_csv(f'{data_path}normalized/ohe/pastHistory.csv', dtype=dtype_dict)
past_hist_df.head()

Remove the uneeded 'Unnamed: 0' column:

In [ ]:
diagns_df = diagns_df.drop(columns='Unnamed: 0')
alrg_df = alrg_df.drop(columns='Unnamed: 0')
past_hist_df = past_hist_df.drop(columns='Unnamed: 0')

### Treatments

In [ ]:
treat_df = pd.read_csv(f'{data_path}normalized/ohe/treatment.csv', dtype=dtype_dict)
treat_df.head()

In [ ]:
adms_drug_df = pd.read_csv(f'{data_path}normalized/ohe/admissionDrug.csv', dtype=dtype_dict)
adms_drug_df.head()

In [ ]:
inf_drug_df = pd.read_csv(f'{data_path}normalized/ohe/infusionDrug.csv', dtype=dtype_dict)
inf_drug_df.head()

In [ ]:
med_df = pd.read_csv(f'{data_path}normalized/ohe/medication.csv', dtype=dtype_dict)
med_df.head()

In [ ]:
in_out_df = pd.read_csv(f'{data_path}normalized/intakeOutput.csv', dtype=dtype_dict)
in_out_df.head()

Remove the uneeded 'Unnamed: 0' column:

In [ ]:
treat_df = treat_df.drop(columns='Unnamed: 0')
adms_drug_df = adms_drug_df.drop(columns='Unnamed: 0')
inf_drug_df = inf_drug_df.drop(columns='Unnamed: 0')
med_df = med_df.drop(columns='Unnamed: 0')
in_out_df = in_out_df.drop(columns='Unnamed: 0')

### Nursing data

In [ ]:
# nurse_care_df = pd.read_csv(f'{data_path}normalized/ohe/nurseCare.csv')
# nurse_care_df.head()

In [ ]:
# nurse_assess_df = pd.read_csv(f'{data_path}normalized/ohe/nurseAssessment.csv')
# nurse_assess_df.head()

Remove the uneeded 'Unnamed: 0' column:

In [ ]:
# nurse_care_df = nurse_care_df.drop(columns='Unnamed: 0')
# nurse_assess_df = nurse_assess_df.drop(columns='Unnamed: 0')

### Respiratory data

In [ ]:
resp_care_df = pd.read_csv(f'{data_path}normalized/ohe/respiratoryCare.csv', dtype=dtype_dict)
resp_care_df.head()

Remove the uneeded 'Unnamed: 0' column:

In [ ]:
resp_care_df = resp_care_df.drop(columns='Unnamed: 0')

### Vital signals

In [ ]:
vital_aprdc_df = pd.read_csv(f'{data_path}normalized/vitalAperiodic.csv', dtype=dtype_dict)
vital_aprdc_df.head()

In [ ]:
vital_prdc_df = pd.read_csv(f'{data_path}normalized/vitalPeriodic.csv', dtype=dtype_dict)
vital_prdc_df.head()

Remove the uneeded 'Unnamed: 0' column:

In [ ]:
vital_aprdc_df = vital_aprdc_df.drop(columns='Unnamed: 0')
vital_prdc_df = vital_prdc_df.drop(columns='Unnamed: 0')

### Exams data

In [ ]:
lab_df = pd.read_csv(f'{data_path}normalized/ohe/lab.csv', dtype=dtype_dict)
lab_df.head()

Remove the uneeded 'Unnamed: 0' column:

In [ ]:
lab_df = lab_df.drop(columns='Unnamed: 0')

## Joining dataframes

### Checking the matching of unit stays IDs

In [ ]:
full_stays_list = set(patient_df.patientunitstayid.unique())

Total number of unit stays:

In [ ]:
len(full_stays_list)

In [ ]:
note_stays_list = set(note_df.patientunitstayid.unique())

In [ ]:
len(note_stays_list)

Number of unit stays that have note data:

In [ ]:
len(set.intersection(full_stays_list, note_stays_list))

In [ ]:
diagns_stays_list = set(diagns_df.patientunitstayid.unique())

In [ ]:
len(diagns_stays_list)

Number of unit stays that have diagnosis data:

In [ ]:
len(set.intersection(full_stays_list, diagns_stays_list))

In [ ]:
alrg_stays_list = set(alrg_df.patientunitstayid.unique())

In [ ]:
len(alrg_stays_list)

Number of unit stays that have allergy data:

In [ ]:
len(set.intersection(full_stays_list, alrg_stays_list))

In [ ]:
past_hist_stays_list = set(past_hist_df.patientunitstayid.unique())

In [ ]:
len(past_hist_stays_list)

Number of unit stays that have past history data:

In [ ]:
len(set.intersection(full_stays_list, past_hist_stays_list))

In [ ]:
treat_stays_list = set(treat_df.patientunitstayid.unique())

In [ ]:
len(treat_stays_list)

Number of unit stays that have treatment data:

In [ ]:
len(set.intersection(full_stays_list, treat_stays_list))

In [ ]:
adms_drug_stays_list = set(adms_drug_df.patientunitstayid.unique())

In [ ]:
len(adms_drug_stays_list)

Number of unit stays that have admission drug data:

In [ ]:
len(set.intersection(full_stays_list, adms_drug_stays_list))

In [ ]:
inf_drug_stays_list = set(inf_drug_df.patientunitstayid.unique())

In [ ]:
len(inf_drug_stays_list)

Number of unit stays that have infusion drug data:

In [ ]:
len(set.intersection(full_stays_list, inf_drug_stays_list))

In [ ]:
med_stays_list = set(med_df.patientunitstayid.unique())

In [ ]:
len(med_stays_list)

Number of unit stays that have medication data:

In [ ]:
len(set.intersection(full_stays_list, med_stays_list))

In [ ]:
in_out_stays_list = set(in_out_df.patientunitstayid.unique())

In [ ]:
len(in_out_stays_list)

Number of unit stays that have intake and output data:

In [ ]:
len(set.intersection(full_stays_list, in_out_stays_list))

In [ ]:
# nurse_care_stays_list = set(nurse_care_df.patientunitstayid.unique())

In [ ]:
# len(nurse_care_stays_list)

Number of unit stays that have nurse care data:

In [ ]:
# len(set.intersection(full_stays_list, nurse_care_stays_list))

In [ ]:
# nurse_assess_stays_list = set(nurse_assess_df.patientunitstayid.unique())

In [ ]:
# len(nurse_assess_stays_list)

Number of unit stays that have nurse assessment data:

In [ ]:
# len(set.intersection(full_stays_list, nurse_assess_stays_list))

In [ ]:
resp_care_stays_list = set(resp_care_df.patientunitstayid.unique())

In [ ]:
len(resp_care_stays_list)

Number of unit stays that have respiratory care data:

In [ ]:
len(set.intersection(full_stays_list, resp_care_stays_list))

In [ ]:
vital_aprdc_stays_list = set(vital_aprdc_df.patientunitstayid.unique())

In [ ]:
len(vital_aprdc_stays_list)

Number of unit stays that have vital aperiodic data:

In [ ]:
len(set.intersection(full_stays_list, vital_aprdc_stays_list))

In [ ]:
vital_prdc_stays_list = set(vital_prdc_df.patientunitstayid.unique())

In [ ]:
len(vital_prdc_stays_list)

Number of unit stays that have vital periodic data:

In [ ]:
len(set.intersection(full_stays_list, vital_prdc_stays_list))

In [ ]:
lab_stays_list = set(lab_df.patientunitstayid.unique())

In [ ]:
len(lab_stays_list)

Number of unit stays that have lab data:

In [ ]:
len(set.intersection(full_stays_list, lab_stays_list))

### Joining patient with note data

In [ ]:
eICU_df = pd.merge(patient_df, note_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining with diagnosis data

Filter to the unit stays that also have data in the other tables:

In [ ]:
diagns_df = diagns_df[diagns_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Also filter only to the unit stays that have data in this new table, considering its importance:

In [ ]:
eICU_df = eICU_df[eICU_df.patientunitstayid.isin(diagns_df.patientunitstayid.unique())]

Merge the dataframes:

In [ ]:
eICU_df = pd.merge(eICU_df, diagns_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining with allergy data

Filter to the unit stays that also have data in the other tables:

In [ ]:
alrg_df = alrg_df[alrg_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Merge the dataframes:

In [ ]:
eICU_df = pd.merge(eICU_df, alrg_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining with past history data

Filter to the unit stays that also have data in the other tables:

In [ ]:
past_hist_df = past_hist_df[past_hist_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Also filter only to the unit stays that have data in this new table, considering its importance:

In [ ]:
eICU_df = eICU_df[eICU_df.patientunitstayid.isin(past_hist_df.patientunitstayid.unique())]

Merge the dataframes:

In [ ]:
len(eICU_df)

In [ ]:
eICU_df = pd.merge(eICU_df, past_hist_df, how='outer', on='patientunitstayid')
eICU_df.head()

In [ ]:
len(eICU_df)

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining with treatment data

Filter to the unit stays that also have data in the other tables:

In [ ]:
treat_df = treat_df[treat_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Also filter only to the unit stays that have data in this new table, considering its importance:

In [ ]:
eICU_df = eICU_df[eICU_df.patientunitstayid.isin(treat_df.patientunitstayid.unique())]

Merge the dataframes:

In [ ]:
eICU_df = pd.merge(eICU_df, treat_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining with admission drug data

Filter to the unit stays that also have data in the other tables:

In [ ]:
adms_drug_df = adms_drug_df[adms_drug_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Merge the dataframes:

In [ ]:
eICU_df = pd.merge(eICU_df, adms_drug_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining with infusion drug data

Filter to the unit stays that also have data in the other tables:

In [ ]:
inf_drug_df = inf_drug_df[inf_drug_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Merge the dataframes:

In [ ]:
eICU_df = pd.merge(eICU_df, inf_drug_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

In [ ]:
eICU_df.patientunitstayid.nunique()

Save the current dataframe:

In [ ]:
eICU_df.to_csv(f'{data_path}normalized/ohe/eICU_before_joining_med.csv')

### Joining with medication data

In [ ]:
eICU_df = pd.read_csv(f'{data_path}normalized/ohe/eICU_before_joining_med.csv',
                      dtype=dtype_dict)
eICU_df = eICU_df.drop(columns=['Unnamed: 0'])
eICU_df.head()

Filter to the unit stays that also have data in the other tables:

In [ ]:
med_df = med_df[med_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Also filter only to the unit stays that have data in this new table, considering its importance:

In [ ]:
eICU_df = eICU_df[eICU_df.patientunitstayid.isin(med_df.patientunitstayid.unique())]

Merge the dataframes:

In [ ]:
eICU_df = pd.merge(eICU_df, med_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining with intake outake data

Filter to the unit stays that also have data in the other tables:

In [ ]:
in_out_df = in_out_df[in_out_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Also filter only to the unit stays that have data in this new table, considering its importance:

In [ ]:
eICU_df = eICU_df[eICU_df.patientunitstayid.isin(in_out_df.patientunitstayid.unique())]

Merge the dataframes:

In [ ]:
eICU_df = pd.merge(eICU_df, in_out_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining with nurse care data

In [ ]:
# eICU_df = pd.merge(eICU_df, nurse_care_df, how='outer', on=['patientunitstayid', 'ts'])
# eICU_df.head()

### Joining with nurse assessment data

In [ ]:
# eICU_df = pd.merge(eICU_df, nurse_assess_df, how='outer', on=['patientunitstayid', 'ts'])
# eICU_df.head()

### Joining with nurse charting data

In [ ]:
# eICU_df = pd.merge(eICU_df, nurse_chart_df, how='outer', on=['patientunitstayid', 'ts'])
# eICU_df.head()

### Joining with respiratory care data

Filter to the unit stays that also have data in the other tables:

In [ ]:
resp_care_df = resp_care_df[resp_care_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Merge the dataframes:

In [ ]:
eICU_df = pd.merge(eICU_df, resp_care_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining with aperiodic vital signals data

Filter to the unit stays that also have data in the other tables:

In [ ]:
vital_aprdc_df = vital_aprdc_df[vital_aprdc_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Also filter only to the unit stays that have data in this new table, considering its importance:

In [ ]:
eICU_df = eICU_df[eICU_df.patientunitstayid.isin(vital_aprdc_df.patientunitstayid.unique())]

Merge the dataframes:

In [ ]:
eICU_df = pd.merge(eICU_df, vital_aprdc_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining with periodic vital signals data

Filter to the unit stays that also have data in the other tables:

In [ ]:
vital_prdc_df = vital_prdc_df[vital_prdc_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Also filter only to the unit stays that have data in this new table, considering its importance:

In [ ]:
eICU_df = eICU_df[eICU_df.patientunitstayid.isin(vital_prdc_df.patientunitstayid.unique())]

Save the current dataframe:

In [ ]:
eICU_df.to_csv(f'{data_path}normalized/ohe/eICU_before_joining_vital_prdc.csv')

Merge the dataframes:

In [ ]:
eICU_df = pd.read_csv(f'{data_path}normalized/ohe/eICU_before_joining_vital_prdc.csv',
                      dtype=dtype_dict)
eICU_df = eICU_df.drop(columns=['Unnamed: 0'])
eICU_df.head()

In [ ]:
eICU_df = pd.merge(eICU_df, vital_prdc_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

Save the current dataframe:

In [ ]:
eICU_df.to_csv(f'{data_path}normalized/ohe/eICU_post_joining_vital_prdc.csv')

In [ ]:
eICU_df = pd.read_csv(f'{data_path}normalized/ohe/eICU_post_joining_vital_prdc.csv',
                      dtype=dtype_dict)
eICU_df = eICU_df.drop(columns=['Unnamed: 0'])
eICU_df.head()

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining with lab data

Filter to the unit stays that also have data in the other tables:

In [ ]:
lab_df = lab_df[lab_df.patientunitstayid.isin(eICU_df.patientunitstayid.unique())]

Also filter only to the unit stays that have data in this new table, considering its importance:

In [ ]:
eICU_df = eICU_df[eICU_df.patientunitstayid.isin(lab_df.patientunitstayid.unique())]

Merge the dataframes:

In [ ]:
eICU_df = pd.merge(eICU_df, lab_df, how='outer', on=['patientunitstayid', 'ts'])
eICU_df.head()

Save the current dataframe:

In [ ]:
eICU_df.to_csv(f'{data_path}normalized/ohe/eICU_post_joining.csv')

In [ ]:
eICU_df = pd.read_csv(f'{data_path}normalized/ohe/eICU_post_joining_0.csv', dtype=dtype_dict)
eICU_df = eICU_df.drop(columns=['Unnamed: 0'])
eICU_df.head()

In [ ]:
eICU_df.columns

In [ ]:
eICU_df.dtypes

In [ ]:
eICU_df.patientunitstayid.nunique()

## Cleaning the joined data

### Removing unit stays that are too short

In [ ]:
# eICU_df.info(memory_usage='deep')

Make sure that the dataframe is ordered by time `ts`:

In [ ]:
eICU_df = eICU_df.sort_values('ts')
eICU_df.head()

Remove unit stays that have less than 10 records:

In [ ]:
unit_stay_len = eICU_df.groupby('patientunitstayid').patientunitstayid.count()
unit_stay_len

In [ ]:
unit_stay_short = set(unit_stay_len[unit_stay_len < 10].index)
unit_stay_short

In [ ]:
len(unit_stay_short)

In [ ]:
eICU_df.patientunitstayid.nunique()

In [ ]:
eICU_df = eICU_df[~eICU_df.patientunitstayid.isin(unit_stay_short)]

In [ ]:
eICU_df.patientunitstayid.nunique()

Remove unit stays that have data that represent less than 48h:

In [ ]:
unit_stay_duration = eICU_df.groupby('patientunitstayid').ts.apply(lambda x: x.max() - x.min())
unit_stay_duration

In [ ]:
unit_stay_short = set(unit_stay_duration[unit_stay_duration < 48*60].index)
unit_stay_short

In [ ]:
len(unit_stay_short)

In [ ]:
eICU_df.patientunitstayid.nunique()

In [ ]:
eICU_df = eICU_df[~eICU_df.patientunitstayid.isin(unit_stay_short)]

In [ ]:
eICU_df.patientunitstayid.nunique()

### Joining duplicate columns

In [ ]:
set([col.split('_x')[0].split('_y')[0] for col in eICU_df.columns if col.endswith('_x') or col.endswith('_y')])

In [ ]:
eICU_df[['drugdosage_x', 'drugdosage_y']].head(20)

In [ ]:
eICU_df[eICU_df.index == 2564878][['drugdosage_x', 'drugdosage_y']]

Convert dataframe to Pandas, as the next cells aren't working properly with Modin:

In [ ]:
eICU_df, pd = du.utils.convert_dataframe(eICU_df, to='pandas', dtypes=dtype_dict)

In [ ]:
eICU_df = du.data_processing.merge_columns(eICU_df, inplace=True)
eICU_df.sample(20)

In [ ]:
set([col.split('_x')[0].split('_y')[0] for col in eICU_df.columns if col.endswith('_x') or col.endswith('_y')])

In [ ]:
eICU_df['drugdosage'].head(20)

In [ ]:
eICU_df[eICU_df.index == 2564878][['drugdosage']]

Save the current dataframe:

In [ ]:
eICU_df.to_csv(f'{data_path}normalized/ohe/eICU_post_merge_duplicate_cols.csv')

In [ ]:
eICU_df = pd.read_csv(f'{data_path}normalized/ohe/eICU_post_merge_duplicate_cols.csv', dtype=dtype_dict)
eICU_df = eICU_df.drop(columns=['Unnamed: 0'])
eICU_df.head()

### Removing unit stays with too many missing values

Consider removing all unit stays that have, combining rows and columns, a very high percentage of missing values.

Reconvert dataframe to Modin:

In [ ]:
eICU_df, pd = du.utils.convert_dataframe(vital_prdc_df, to='modin', dtypes=dtype_dict)

In [ ]:
n_features = len(eICU_df.columns)
n_features

Create a temporary column that counts each row's number of missing values:

In [ ]:
eICU_df['row_msng_val'] = eICU_df.isnull().sum(axis=1)
eICU_df[['patientunitstayid', 'ts', 'row_msng_val']].head()

Check each unit stay's percentage of missing data points:

In [ ]:
# Number of possible data points in each unit stay
n_data_points = eICU_df.groupby('patientunitstayid').ts.count() * n_features
n_data_points

In [ ]:
# Number of missing values in each unit stay
n_msng_val = eICU_df.groupby('patientunitstayid').row_msng_val.sum()
n_msng_val

In [ ]:
# Percentage of missing values in each unit stay
msng_val_prct = (n_msng_val / n_data_points) * 100
msng_val_prct

In [ ]:
msng_val_prct.describe()

Remove unit stays that have too many missing values (>70% of their respective data points):

In [ ]:
unit_stay_high_msgn = set(msng_val_prct[msng_val_prct > 70].index)
unit_stay_high_msgn

In [ ]:
eICU_df.patientunitstayid.nunique()

In [ ]:
eICU_df = eICU_df[~eICU_df.patientunitstayid.isin(unit_stay_high_msgn)]

In [ ]:
eICU_df.patientunitstayid.nunique()

### Removing columns with too many missing values

We should remove features that have too many missing values (in this case, those that have more than 40% of missing values). Without enough data, it's even risky to do imputation, as it's unlikely for the imputation to correctly model the missing feature.

In [ ]:
du.search_explore.dataframe_missing_values(eICU_df)

In [ ]:
prev_features = eICU_df.columns
len(prev_features)

In [ ]:
eICU_df = du.data_processing.remove_cols_with_many_nans(eICU_df, nan_percent_thrsh=70, inplace=True)

In [ ]:
features = eICU_df.columns
len(features)

Removed features:

In [ ]:
set(prev_features) - set(features)

In [ ]:
eICU_df.head()

### Performing imputation

In [ ]:
du.search_explore.dataframe_missing_values(eICU_df)

In [ ]:
eICU_df = du.data_processing.missing_values_imputation(eICU_df, method='interpolation',
                                                       id_column='patientunitstay', inplace=True)
eICU_df.head()

In [ ]:
du.search_explore.dataframe_missing_values(eICU_df)

## Setting the label

Define the label column considering the desired time window on which we want to predict mortality (0, 24h, 48h, 72h, etc).

In [ ]:
time_window_h = 24

In [ ]:
eICU_df['label'] = eICU_df[eICU_df.death_ts - eICU_df.ts <= time_window_h * 60]
eICU_df.head()

## Creating a single encoding dictionary for the complete dataframe

Combine the one hot encoding dictionaries of all tables, having in account the converged ones, into a single dictionary representative of all the categorical features in the resulting dataframe.

In [ ]:
stream_adms_drug = open(f'{data_path}cat_feat_ohe_adms_drug.yaml', 'r')
stream_inf_drug = open(f'{data_path}cat_feat_ohe_inf_drug.yaml', 'r')
stream_med = open(f'{data_path}cat_feat_ohe_med.yaml', 'r')
stream_treat = open(f'{data_path}cat_feat_ohe_treat.yaml', 'r')
stream_diag = open(f'{data_path}cat_feat_ohe_diag.yaml', 'r')
stream_alrg = open(f'{data_path}cat_feat_ohe_alrg.yaml', 'r')
stream_past_hist = open(f'{data_path}cat_feat_ohe_past_hist.yaml', 'r')
stream_lab = open(f'{data_path}cat_feat_ohe_lab.yaml', 'r')
stream_patient = open(f'{data_path}cat_feat_ohe_patient.yaml', 'r')
stream_notes = open(f'{data_path}cat_feat_ohe_note.yaml', 'r')

In [ ]:
cat_feat_ohe_adms_drug = yaml.load(stream_adms_drug, Loader=yaml.FullLoader)
cat_feat_ohe_inf_drug = yaml.load(stream_inf_drug, Loader=yaml.FullLoader)
cat_feat_ohe_med = yaml.load(stream_med, Loader=yaml.FullLoader)
cat_feat_ohe_treat = yaml.load(stream_treat, Loader=yaml.FullLoader)
cat_feat_ohe_diag = yaml.load(stream_diag, Loader=yaml.FullLoader)
cat_feat_ohe_alrg = yaml.load(stream_alrg, Loader=yaml.FullLoader)
cat_feat_ohe_past_hist = yaml.load(stream_past_hist, Loader=yaml.FullLoader)
cat_feat_ohe_lab = yaml.load(stream_lab, Loader=yaml.FullLoader)
cat_feat_ohe_patient = yaml.load(stream_patient, Loader=yaml.FullLoader)
cat_feat_ohe_notes = yaml.load(stream_notes, Loader=yaml.FullLoader)

In [ ]:
cat_feat_ohe = du.utils.merge_dicts([cat_feat_ohe_adms_drug, cat_feat_ohe_inf_drug,
                                     cat_feat_ohe_med, cat_feat_ohe_treat,
                                     cat_feat_ohe_diag, cat_feat_ohe_alrg,
                                     cat_feat_ohe_past_hist, cat_feat_ohe_lab,
                                     cat_feat_ohe_patient, cat_feat_ohe_notes])

In [ ]:
list(cat_feat_ohe.keys())

Clean the one hot encoded column names, as they are in the final dataframe:

In [ ]:
for key, val in cat_feat_ohe.items():
    cat_feat_ohe[key] = du.data_processing.clean_naming(cat_feat_ohe[key], lower_case=False)

Save the final encoding dictionary:

In [ ]:
stream = open(f'{data_path}/cleaned/cat_feat_ohe_eICU.yaml', 'w')
yaml.dump(cat_feat_ohe, stream, default_flow_style=False)

## Creating a single normalization dictionary for the complete dataframe

Combine the normalization stats dictionaries of all tables into a single dictionary representative of all the continuous features in the resulting dataframe.

In [ ]:
stream_adms_drug = open(f'{data_path}admissionDrug_norm_stats.yaml', 'r')
stream_inf_drug = open(f'{data_path}infusionDrug_norm_stats.yaml', 'r')
stream_med = open(f'{data_path}medication_norm_stats.yaml', 'r')
# stream_in_out = open(f'{data_path}cat_embed_feat_enum_in_out.yaml', 'r')
stream_lab = open(f'{data_path}lab_norm_stats.yaml', 'r')
stream_patient = open(f'{data_path}patient_norm_stats.yaml', 'r')
# stream_vital_aprdc = open(f'{data_path}vitalAperiodic_norm_stats.yaml', 'r')
# stream_vital_prdc = open(f'{data_path}vitalPeriodic_norm_stats.yaml', 'r')

In [ ]:
norm_stats_adms_drug = yaml.load(stream_adms_drug, Loader=yaml.FullLoader)
norm_stats_inf_drug = yaml.load(stream_inf_drug, Loader=yaml.FullLoader)
norm_stats_med = yaml.load(stream_med, Loader=yaml.FullLoader)
# norm_stats__in_out = yaml.load(stream_in_out, Loader=yaml.FullLoader)
norm_stats_lab = yaml.load(stream_lab, Loader=yaml.FullLoader)
norm_stats_patient = yaml.load(stream_patient, Loader=yaml.FullLoader)
# norm_stats_vital_aprdc = yaml.load(stream_vital_aprdc, Loader=yaml.FullLoader)
# norm_stats_vital_prdc = yaml.load(stream_vital_prdc, Loader=yaml.FullLoader)

In [ ]:
norm_stats = du.utils.merge_dicts([norm_stats_adms_drug, norm_stats_inf_drug,
                                   norm_stats_med,
#                                    norm_stats_in_out, 
                                   norm_stats_lab, norm_stats_patient, 
                                   norm_stats_vital_aprdc, norm_stats_vital_prdc])

In [ ]:
list(norm_stats.keys())

Save the final encoding dictionary:

In [ ]:
stream = open(f'{data_path}/cleaned/eICU_norm_stats.yaml', 'w')
yaml.dump(norm_stats, stream, default_flow_style=False)